In [4]:
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218921 sha256=f618fd9b72f0ed647fa55be707580c67594dc28ff6a13e9be487555fe5e278cb
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [2]:
!pip install vectorbt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.2 MB/s eta 0:00:00


In [5]:
import numpy as  np
import pandas as pd
import vectorbt as vbt
import pandas_ta as ta
import yfinance as yf

In [9]:
symbols = ['USDJPY=x']
start_date = '1996-10-30'
end_date = '2001-10-30'
price = vbt.YFData.download(symbols,start=start_date,end=end_date,interval='1D',missing_index='drop')
price = price.loc[(price.wrapper.index >= start_date) & (price.wrapper.index < end_date)]

In [11]:
df = pd.DataFrame()
df = price.data[symbols[0]]
df.drop(['Volume','Dividends','Stock Splits'],axis=1,inplace=True)

In [12]:
df.index = pd.to_datetime(df.index).strftime('%Y-%m-%d')

In [13]:
BB = ta.bbands(df.Close,length=14,std =2,mamode="ema")

In [19]:
Entry = (df.Close<BB["BBL_14_2.0"])
Exit = (df.Close>BB["BBU_14_2.0"])
Entry = Entry.shift(1)
Exit = Exit.shift(1)
Entry.fillna(False,inplace=True)
Exit.fillna(False,inplace=True)
Entry_signal_vectorbt = ta.tsignals(Entry,asbool=True,appned=True)
Exit_signal_vectorbt = ta.tsignals(Exit,asbool=True,append=True)

In [20]:
port = vbt.Portfolio.from_signals(df.Close,
                                  entries = Entry_signal_vectorbt.TS_Entries,
                                  exits = Exit_signal_vectorbt.TS_Entries,
                                  size = 30,
                                  sl_stop = 0.05,
                                  sl_trail = True,
                                  freq = 'D',
                                  init_cash = 10000,
                                  fees = 0.0025,
                                  slippage = 0.0025)

In [21]:
port.plot().show()